In [1]:
import os
import json
import shutil
import numpy as np

# SCRIPT PARAMS #

In [2]:
# Output directory
OUTPUT_DIR = 'models_reordered'
if os.path.isdir(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.mkdir(OUTPUT_DIR)

# Models
ALL_MODELS = ['pointnet', 'pointnet++', 'so-net', 'kcnet', 'dgcnn', 'pointcnn'] #'deepsets',

# Class names
CLASSES_COUNT = 40
CLASS_NAMES = ['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl',
               'car', 'chair', 'cone', 'cup', 'curtain', 'desk', 'door', 'dresser',
               'flower_pot', 'glass_box', 'guitar', 'keyboard', 'lamp', 'laptop',
               'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 'radio',
               'range_hood', 'sink', 'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet',
               'tv_stand', 'vase', 'wardrobe', 'xbox']

# Test class instances
CLASS_TEST_INSTANCES = [100, 50, 100, 20, 100, 100, 20, 100, 100, 20, 20, 20, 86, 20,
                        86, 20, 100, 100, 20, 20, 20, 100, 100, 86, 20, 100, 100, 20,
                        100, 20, 100, 20, 20, 100, 20, 100, 100, 100, 20, 20]

# Ensemble params
MODELS_COUNT = 10
ENSEMBLE_MODELS_NUMBER = 10

# LOAD PROBS #

In [3]:
probabilities = {}
true_labels = {}
probabilities_reordered = {}
true_labels_reordered = {}
for model in ALL_MODELS:
    
    # Load and transpose
    model_filepath = os.path.join('models_raw', model)
    prob_filepath = os.path.join(model_filepath, 'probabilities.npy')
    probabilities[model] = np.load(prob_filepath)
    probabilities[model] = np.transpose(probabilities[model], axes=(2, 0, 1))

    # probabilities[model] =  softmax(probabilities[model])
    probabilities[model] = probabilities[model]/np.std(probabilities[model])

    label_filepath = os.path.join(model_filepath, 'true_labels.npy')
    true_labels[model] = np.load(label_filepath)
    for class_idx in range(CLASSES_COUNT):
        error_info = ("Test cloud instances for model " + model + " and class "
                      + str(class_idx) + " differs! Should be " + str(CLASS_TEST_INSTANCES[class_idx])
                      + " but got " + str(np.sum(true_labels[model] == class_idx)))
        assert np.sum(true_labels[model] == class_idx) == CLASS_TEST_INSTANCES[class_idx] or \
          (model=='deepsets' and class_idx==12 and np.sum(true_labels[model] == class_idx) == 85) or \
          (model=='deepsets' and class_idx==39 and np.sum(true_labels[model] == class_idx) == 19), \
          error_info
                
    assert (probabilities[model].shape[0] == ENSEMBLE_MODELS_NUMBER)
    assert (probabilities[model].shape[1] == 2468 or (model=='deepsets' and probabilities[model].shape[1] == 2466))
    assert (probabilities[model].shape[2] == CLASSES_COUNT)

## SO-Net

In [4]:
if 'so-net' in ALL_MODELS:
    sonet_list_path = 'models_raw/so-net/modelnet40_test.txt'
    sonet_list = []
    with open(sonet_list_path) as f:
        for row in f:
            sonet_list.append(row[:-1])
    probabilities_reordered['so-net'] = probabilities['so-net']
    true_labels_reordered['so-net'] = true_labels['so-net']

## PointNet

In [5]:
if 'pointnet' in ALL_MODELS:
    pointnet_list_path = \
        ['models_raw/pointnet/ply_data_test_0_id2file.json',
         'models_raw/pointnet/ply_data_test_1_id2file.json']
    pointnet_list = []
    for filepath in pointnet_list_path:
        with open(filepath) as f:
            data = json.load(f)
            pointnet_list += data
    pointnet_list = [(i, pointnet_list[i].split('/')[1][:-4]) for i in range(len(pointnet_list))]
    pointnet_list = sorted(pointnet_list, key=lambda x: x[1])

    # Size assert wrt SO-Net
    assert(len(pointnet_list) == len(sonet_list))
    for i in range(len(sonet_list)):
        assert(sonet_list[i] == pointnet_list[i][1])

 ## PointNet++

In [6]:
if 'pointnet++' in ALL_MODELS:
    pointnetpp_list_path = \
        ['models_raw/pointnet++/ply_data_test_0_id2file.json',
         'models_raw/pointnet++/ply_data_test_1_id2file.json']
    pointnetpp_list = []
    for filepath in pointnetpp_list_path:
        with open(filepath) as f:
            data = json.load(f)
            pointnetpp_list += data
    pointnetpp_list = [(i, pointnetpp_list[i].split('/')[1][:-4]) for i in range(len(pointnetpp_list))]
    pointnetpp_list = sorted(pointnetpp_list, key=lambda x: x[1])

    # Size assert wrt SO-Net
    assert(len(pointnetpp_list) == len(sonet_list))
    for i in range(len(sonet_list)):
        assert(sonet_list[i] == pointnetpp_list[i][1])

## DGCNN

In [7]:
if 'dgcnn' in ALL_MODELS:
    dgcnn_list_path = \
        ['models_raw/dgcnn/ply_data_test_0_id2file.json',
         'models_raw/dgcnn/ply_data_test_1_id2file.json']
    dgcnn_list = []
    for filepath in dgcnn_list_path:
        with open(filepath) as f:
            data = json.load(f)
            dgcnn_list += data
    dgcnn_list = [(i, dgcnn_list[i].split('/')[1][:-4]) for i in range(len(dgcnn_list))]
    dgcnn_list = sorted(dgcnn_list, key=lambda x: x[1])

    # Size assert wrt SO-Net
    assert(len(dgcnn_list) == len(sonet_list))
    for i in range(len(sonet_list)):
        assert(sonet_list[i] == dgcnn_list[i][1])

## PointCNN

In [8]:
if 'pointcnn' in ALL_MODELS:
    pointcnn_list_path = \
        ['models_raw/pointcnn/ply_data_test_0_id2file.json',
         'models_raw/pointcnn/ply_data_test_1_id2file.json']
    pointcnn_list = []
    for filepath in pointcnn_list_path:
        with open(filepath) as f:
            data = json.load(f)
            pointcnn_list += data
    pointcnn_list = [(i, pointcnn_list[i].split('/')[1][:-4]) for i in range(len(pointcnn_list))]
    pointcnn_list = sorted(pointcnn_list, key=lambda x: x[1])

    # Size assert wrt SO-Net
    assert(len(pointcnn_list) == len(sonet_list))
    for i in range(len(sonet_list)):
        assert(sonet_list[i] == pointcnn_list[i][1])

## Rearange pointnet/pointnet++/dgcnn

In [9]:
# Reorder pointnet
if 'pointnet' in ALL_MODELS:
    pointnet_indices = [x[0] for x in pointnet_list]
    probabilities_reordered['pointnet'] = probabilities['pointnet'][:, pointnet_indices, :]
    true_labels_reordered['pointnet'] = true_labels['pointnet'][pointnet_indices]
    assert((true_labels_reordered['pointnet'] == true_labels_reordered['so-net']).all())

# Reorder pointnet++
if 'pointnet++' in ALL_MODELS:
    pointnetpp_indices = [x[0] for x in pointnetpp_list]
    probabilities_reordered['pointnet++'] = probabilities['pointnet++'][:, pointnetpp_indices, :]
    true_labels_reordered['pointnet++'] = true_labels['pointnet++'][pointnetpp_indices]
    assert((true_labels_reordered['pointnet++'] == true_labels_reordered['so-net']).all())

# Reorder dgcnn
if 'dgcnn' in ALL_MODELS:
    dgcnn_indices = [x[0] for x in dgcnn_list]
    probabilities_reordered['dgcnn'] = probabilities['dgcnn'][:, dgcnn_indices, :]
    true_labels_reordered['dgcnn'] = true_labels['dgcnn'][dgcnn_indices]
    assert((true_labels_reordered['dgcnn'] == true_labels_reordered['so-net']).all())

# Reorder pointcnn
if 'pointcnn' in ALL_MODELS:
    pointcnn_indices = [x[0] for x in pointcnn_list]
    probabilities_reordered['pointcnn'] = probabilities['pointcnn'][:, pointcnn_indices, :]
    true_labels_reordered['pointcnn'] = true_labels['pointcnn'][pointcnn_indices]
    assert((true_labels_reordered['pointcnn'] == true_labels_reordered['so-net']).all())

## KCNet

In [10]:
xdata_pointnet = np.load('models_raw/pointnet/xdata.npy')[pointnet_indices, :, :]
if 'kcnet' in ALL_MODELS:
    xdata_kcnet = np.load('models_raw/kcnet/xdata.npy')

In [11]:
if 'kcnet' in ALL_MODELS:
    kcnet_indices = []
    for i in range(xdata_pointnet.shape[0]):
        sample_pointnet = xdata_pointnet[i, :, :]
        hits = 0
        for j in range(xdata_kcnet.shape[0]):
            sample_kcnet = xdata_kcnet[j, :, :]
            if np.equal(sample_pointnet, sample_kcnet).all():
                kcnet_indices.append(j)
                hits += 1
        assert(hits == 1)

In [12]:
if 'kcnet' in ALL_MODELS:
    probabilities_reordered['kcnet'] = probabilities['kcnet'][:, kcnet_indices, :]
    true_labels_reordered['kcnet'] = true_labels['kcnet'][kcnet_indices]
    assert((true_labels_reordered['kcnet'] == true_labels_reordered['so-net']).all())

## DeepSets

In [13]:
if 'deepsets' in ALL_MODELS:
    xdata_deepsets = np.load('/home/chechli/ModelNet/DeepSets/DeepSets/PointClouds/xdata.npy')

In [14]:
if 'deepsets' in ALL_MODELS:
    import shutil
    matching_basepath = '/home/chechli/ModelNet/DeepSets/matching'
    if os.path.exists(matching_basepath):
        shutil.rmtree(matching_basepath)
    os.mkdir(matching_basepath)

    for i in range(CLASSES_COUNT):
        class_dir = os.path.join(matching_basepath, 'class_' + str(i))
        os.mkdir(class_dir)
        pointnet_path = os.path.join(class_dir, 'pointnet')
        os.mkdir(pointnet_path)
        deepsets_path = os.path.join(class_dir, 'deepsets')
        os.mkdir(deepsets_path)

        deep_sets_samples_list = []
        for k in range(true_labels['deepsets'].shape[0]):
            if true_labels['deepsets'][k] == i:
                deep_sets_samples_list.append(k)
        pointnet_samples_list = []
        for k in range(true_labels['pointnet'].shape[0]):
            if true_labels['pointnet'][k] == i:
                pointnet_samples_list.append(k)

        for j in pointnet_samples_list:
            sample_pointnet = xdata_pointnet[j, :, :]
            sample_path = os.path.join(pointnet_path, 'sample_' + str(j) + '.npy')
            np.save(sample_path, sample_pointnet)
        for j in deep_sets_samples_list: #xdata_deepsets.shape[0]):
            sample_deepsets = xdata_deepsets[j, :, :]
            sample_path = os.path.join(deepsets_path, 'sample_' + str(j) + '.npy')
            np.save(sample_path, sample_deepsets)
        print('Processed class', i)

## Save results

In [15]:
for model_name in probabilities_reordered:
    base_path = os.path.join(OUTPUT_DIR, model_name)
    os.mkdir(base_path)
    np.save(os.path.join(base_path, 'probabilities.npy'), probabilities_reordered[model_name])
    np.save(os.path.join(base_path, 'true_labels.npy'), true_labels_reordered[model_name])